In [27]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import os
import warnings
import csv
from sklearn.linear_model import LassoCV
from stg import STG
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm



In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Suppress all warnings
warnings.filterwarnings('ignore')


In [29]:
model_used = 'Neural_Network'
print_interval = 5000
epochs = 10000
selection_method = 'Stochastic Gates'
# selection_method = 'lasso'

Light_GBM_global = [
    # "flair_t1ce_t2",
    # "flair_t1ce_t2",
    # "flair",
    "flair_t1ce",
    # "flair_t1ce"
]
Light_GBM_local = [
    # "flair_t1ce_t2",
    "flair_t1ce",
    # "flair",
    # "t2",
    # "flair_t1ce",
]


In [30]:
def ensure_directory_exists(filepath):
    os.makedirs(os.path.dirname(filepath), exist_ok=True)


In [31]:
def load_radiomic_fetures(target_directory, file_name):
    # Load the numpy array from the file in the target directory
    file_path = os.path.join(target_directory, file_name)
    array = np.load(file_path)
    print(f"Array loaded from '{file_path}'")
    return array

In [32]:
def load_model_and_parameters(modality_used,  model_class):
    model_dir = f"./models/{selection_method}_feature_selection/{model_used}/{modality_used}/"
    
    model_file = os.path.join(model_dir, 'model.pt')
    params_file = os.path.join(model_dir, 'params.txt')
    mask_file = os.path.join(model_dir, f'{selection_method}_mask.npy')
    # Load the selected features mask
    mask = np.load(mask_file)
    inputdim = np.count_nonzero(mask)
    # Load the model
    model = model_class(inputdim)
    model.load_state_dict(torch.load(model_file))
    model.eval()  # Set the model to evaluation mode
    
    # Load the parameters
    with open(params_file, 'r') as file:
        params = file.read().strip()
    
    
    print(f"Model, parameters, and {selection_method} mask loaded successfully for modality {modality_used}.")
    
    return model, params, mask

In [33]:
def load_and_combine_features(modality_keys, dataset_type):
    combined_features = []
    for modality in modality_keys:
        # Load the features for each modality
        features = np.load(f'../local_spatial_Framework/features/{modality}/{dataset_type}/{dataset_type}_backbone_outputs.npy')
        combined_features.append(features)
    # Combine features along the feature dimension (axis=1)
    return np.concatenate(combined_features, axis=1)


In [34]:

def make_csv(y_pred_validation, modality_used):
    df = pd.read_csv('../dataset/MICCAI_BraTS2020_ValidationData/survival_evaluation.csv')
    validation_ids = df['BraTS20ID'].values
    
    #removing 116 paitents ID from list
    validation_ids = np.delete(validation_ids, 26, axis=0)
    
    filename = f"../radiomics_local_global_predictions/{selection_method}_feture_selection/{model_used}/{modality_used}_{model_used}_verifying_again.csv"

    ensure_directory_exists(filename)

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Days"])
        for id, day in zip(validation_ids, y_pred_validation):
            writer.writerow([id, day])

    print(f"CSV file '{filename}' created successfully.")


In [35]:
def load_features(modality_used):
    base_dir = os.path.join('../Global_extracted_features', modality_used)
    train_features = np.load(os.path.join(base_dir, 'train_features.npy'))
    validate_features = np.load(os.path.join(base_dir, 'validate_features.npy'))
    train_labels = np.load(os.path.join(base_dir, 'train_labels.npy'))
    return train_features, validate_features, train_labels


In [36]:
# 2nd model
# class SimpleNN(nn.Module):
#     def __init__(self, input_dim):
#         super(SimpleNN, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 1)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


In [37]:
# 1st model
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [38]:

def do_prediction(train_features, test_features, train_labels, modality_used):
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    test_features_scaled = scaler.transform(test_features)
    print(f"Size of features before Lasso: {train_features_scaled.shape}")
    
    model, param, mask = load_model_and_parameters(modality_used, SimpleNN)
    train_features_selected = train_features_scaled[:, mask]
    test_features_selected = test_features_scaled[:, mask]

    print(f"Size of features after Stochastic Gates: {train_features_selected.shape}")
      
    # Split the selected features into training and validation sets
    train_features_final, val_features_final, train_labels_final, val_labels_final = train_test_split(
        train_features_selected, train_labels, test_size=0.2, random_state=42)

    # Convert features and labels to PyTorch tensors
    train_features_tensor = torch.tensor(train_features_final, dtype=torch.float32).to(device)
    train_labels_tensor = torch.tensor(train_labels_final, dtype=torch.float32).view(-1, 1).to(device)
    val_features_tensor = torch.tensor(val_features_final, dtype=torch.float32).to(device)
    val_labels_tensor = torch.tensor(val_labels_final, dtype=torch.float32).view(-1, 1).to(device)
    test_features_tensor = torch.tensor(test_features_selected, dtype=torch.float32).to(device)

    # Create DataLoader for training data
    train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Make predictions on the test set
    model.eval()
    with torch.no_grad():
        y_pred_test = model(test_features_tensor).cpu().numpy().flatten()

    make_csv(y_pred_test, modality_used)

In [39]:

radiomic_Train_fetures = load_radiomic_fetures("../radiomics features/all", "radiomics_train.npy")
radiomic_Validation_fetures = load_radiomic_fetures("../radiomics features/all", "radiomics_validate.npy")

print("radiomic_Train_fetures",radiomic_Train_fetures.shape)
print("radiomic_Validation_fetures",radiomic_Validation_fetures.shape)

Array loaded from '../radiomics features/all/radiomics_train.npy'
Array loaded from '../radiomics features/all/radiomics_validate.npy'
radiomic_Train_fetures (235, 400)
radiomic_Validation_fetures (28, 400)


In [40]:
for modality_used_global, modality_used_local in zip(Light_GBM_global, Light_GBM_local):
    modality_key_local = modality_used_local.split("_")
    modality_keys_list_global_features = modality_used_global.split("_")
    print(f"\nLoading and combining features... \n local-{modality_used_local}\n global-{modality_used_global}")

    local_train_features = load_and_combine_features(modality_key_local, 'train')
    local_validation_features = load_and_combine_features(modality_key_local, 'validation')
    
    global_train_features, global_validate_features, train_labels = load_features(modality_used_global)
    
    print("global_train_features",global_train_features.shape, "local_train_features", local_train_features.shape)
    print("global_validate_features", global_validate_features.shape, "local_validation_features",local_validation_features.shape)


    local_global_training_features = np.concatenate((global_train_features, local_train_features), axis=1)
    local_global_validation_features = np.concatenate((global_validate_features, local_validation_features), axis=1)
    
#   To remove the data of absent paitents from the local and global features
    train_labels = np.delete(train_labels, 98, axis=0)
    local_global_training_features = np.delete(local_global_training_features, 98, axis=0)
    local_global_validation_features = np.delete(local_global_validation_features, 26, axis=0)


    training_all_features = np.concatenate((local_global_training_features, radiomic_Train_fetures), axis=1)
    validation_all_features = np.concatenate((local_global_validation_features,radiomic_Validation_fetures), axis=1)

    print("combining all")
    print("radiomics_local_global_training",training_all_features.shape)
    print("radiomics_local_global_Validation",validation_all_features.shape)

    modality_used = 'global_' + modality_used_global + '___local_' + modality_used_local
    do_prediction(training_all_features, validation_all_features, train_labels, modality_used)


Loading and combining features... 
 local-flair_t1ce
 global-flair_t1ce
global_train_features (236, 744) local_train_features (236, 128)
global_validate_features (29, 744) local_validation_features (29, 128)
combining all
radiomics_local_global_training (235, 1272)
radiomics_local_global_Validation (28, 1272)
Size of features before Lasso: (235, 1272)
Model, parameters, and Stochastic Gates mask loaded successfully for modality global_flair_t1ce___local_flair_t1ce.
Size of features after Stochastic Gates: (235, 532)


CSV file '../radiomics_local_global_predictions/Stochastic Gates_feture_selection/Neural_Network/global_flair_t1ce___local_flair_t1ce_Neural_Network_verifying_again.csv' created successfully.
